In [1]:
import random
import time
import pulp
import numpy as np
import pandas as pd
from collections import defaultdict

# Set random seed for reproducibility
random.seed(42)

# Instance generation function
def generate_set_cover_instance(U_size, num_subsets, min_weight, max_weight, min_set_size, max_set_size):
    universe = set(range(1, U_size + 1))
    subsets = []
    weights = []
    universe_list = sorted(universe)  # Convert set to list for sampling
    for _ in range(num_subsets):
        set_size = random.randint(min_set_size, max_set_size)
        subset = set(random.sample(universe_list, set_size))
        weight = random.uniform(min_weight, max_weight)
        subsets.append(subset)
        weights.append(weight)
    return universe, subsets, weights

# Greedy set cover function
def greedy_set_cover(universe, subsets, weights):
    covered = set()
    total_weight = 0
    subsets_remaining = list(zip(subsets, weights, range(len(subsets))))
    while covered != universe:
        best_ratio = float('inf')
        best_subset, best_weight, best_index = None, None, None
        for subset, weight, index in subsets_remaining:
            uncovered_elements = subset - covered
            if uncovered_elements:
                ratio = weight / len(uncovered_elements)
                if ratio < best_ratio:
                    best_ratio = ratio
                    best_subset, best_weight, best_index = subset, weight, index
        if best_subset is None:
            break
        total_weight += best_weight
        covered.update(best_subset)
        subsets_remaining = [s for s in subsets_remaining if s[2] != best_index]
    return total_weight, len(covered)

# Optimal set cover function
def optimal_set_cover(universe, subsets, weights):
    prob = pulp.LpProblem('SetCover', pulp.LpMinimize)
    x = [pulp.LpVariable(f'x_{i}', cat='Binary') for i in range(len(subsets))]
    prob += pulp.lpSum([weights[i] * x[i] for i in range(len(subsets))])
    for element in universe:
        prob += pulp.lpSum([x[i] if element in subsets[i] else 0 for i in range(len(subsets))]) >= 1
    prob.solve()
    total_weight = sum(weights[i] for i in range(len(subsets)) if x[i].varValue == 1.0)
    covered_elements = {e for i, subset in enumerate(subsets) if x[i].varValue == 1.0 for e in subset}
    return total_weight, len(covered_elements)

# Simulation function to run multiple instances
def simulate_set_cover(U_size_list, num_subsets_list, min_weight, max_weight, min_set_size, max_set_size, num_trials=5):
    results = defaultdict(list)

    for U_size in U_size_list:
        for num_subsets in num_subsets_list:
            for trial in range(num_trials):
                universe, subsets, weights = generate_set_cover_instance(U_size, num_subsets, min_weight, max_weight, min_set_size, max_set_size)
                
                # Greedy solution
                start_time = time.time()
                greedy_weight, greedy_covered = greedy_set_cover(universe, subsets, weights)
                greedy_time = time.time() - start_time
                
                # Optimal solution (only for small instances)
                if U_size <= 100 and num_subsets <= 50:
                    start_time = time.time()
                    optimal_weight, optimal_covered = optimal_set_cover(universe, subsets, weights)
                    optimal_time = time.time() - start_time
                else:
                    optimal_weight, optimal_covered, optimal_time = None, None, None

                # Store results
                results['U_size'].append(U_size)
                results['num_subsets'].append(num_subsets)
                results['trial'].append(trial + 1)
                results['greedy_weight'].append(greedy_weight)
                results['greedy_covered'].append(greedy_covered)
                results['greedy_time'].append(greedy_time)
                results['optimal_weight'].append(optimal_weight)
                results['optimal_covered'].append(optimal_covered)
                results['optimal_time'].append(optimal_time)

    return results

# Run the simulation
U_size_list = [10, 50, 100, 200, 500]  # Increasing universe sizes
num_subsets_list = [10, 25, 50, 75, 100]  # Vary the number of subsets
min_weight, max_weight = 1, 10
min_set_size, max_set_size = 1, 5

results = simulate_set_cover(U_size_list, num_subsets_list, min_weight, max_weight, min_set_size, max_set_size)

# Display results
results_df = pd.DataFrame(results)
print(results_df)

# Analyze the results
greedy_limit = results_df[results_df['greedy_time'] < 1.0]  # Find cases where greedy time < 1 second
optimal_limit = results_df[results_df['optimal_time'].notnull() & (results_df['optimal_time'] < 1.0)]  # Optimal solution time < 1 sec

print("\nGreedy Limit Analysis (Greedy Time < 1 sec):")
print(greedy_limit.groupby(['U_size', 'num_subsets']).size())

print("\nOptimal Limit Analysis (Optimal Time < 1 sec):")
print(optimal_limit.groupby(['U_size', 'num_subsets']).size())


     U_size  num_subsets  trial  greedy_weight  greedy_covered  greedy_time  \
0        10           10      1      26.400378              10     0.000000   
1        10           10      2      12.326495               9     0.000000   
2        10           10      3      25.705312              10     0.000000   
3        10           10      4      22.646218              10     0.000000   
4        10           10      5      19.372225              10     0.000000   
..      ...          ...    ...            ...             ...          ...   
120     500          100      1     456.036579             219     0.003379   
121     500          100      2     525.007810             233     0.000439   
122     500          100      3     499.094029             224     0.000000   
123     500          100      4     521.693424             212     0.000000   
124     500          100      5     413.846396             210     0.000000   

     optimal_weight  optimal_covered  optimal_time 